In [ ]:
!pip install accelerate datasets rouge_score wandb bitsandbytes accelerate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62

In [ ]:
import os
import nltk
import random

import torch
import numpy as np

import datasets
import transformers
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from tabulate import tabulate
import wandb

In [ ]:
ENCODER_MAX_LENGTH = 16
DECODER_MAX_LENGTH = 64
MODEL_NAME = "kykim/bertshared-kor-base"

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/4.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/589M [00:00<?, ?B/s]

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [ ]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
data = datasets.load_dataset("json", data_files='70B_dataset_modified.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def flatten(example):
   product = example['상품']
   value = example['담고자 하는 가치'].strip('"')
   copy = example['카피라이팅'].strip('"')
   input = product + '<SEP>' + value
   return {'copy':copy, 'input':input}

In [ ]:
data = data.map(flatten, remove_columns=['상품','담고자 하는 가치','카피라이팅'])

Map:   0%|          | 0/10478 [00:00<?, ? examples/s]

In [ ]:
data = data['train']
data = data.train_test_split(test_size=0.1)

In [ ]:
train_data_txt = data['train']
val_data_txt = data['test']

In [ ]:
train_data_txt[1]

{'copy': '손쉽게 편안함을 찾아드립니다', 'input': '요통 완화 벨트<SEP>직관적인'}

In [ ]:
special_tokens_dict = {'additional_special_tokens': ['<SEP>']}
tokenizer.add_special_tokens(special_tokens_dict)

1

In [ ]:
model.encoder.resize_token_embeddings(len(tokenizer))
model.decoder.resize_token_embeddings(len(tokenizer))

Embedding(42001, 768)

In [ ]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["input"], batch["copy"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, ENCODER_MAX_LENGTH, DECODER_MAX_LENGTH
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = val_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, ENCODER_MAX_LENGTH, DECODER_MAX_LENGTH
    ),
    batched=True,
    remove_columns=val_data_txt.column_names,
)

Map:   0%|          | 0/9430 [00:00<?, ? examples/s]

Map:   0%|          | 0/1048 [00:00<?, ? examples/s]

In [ ]:
nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

<ipython-input-16-9f91a8670018>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
os.environ["WANDB_PROJECT"]="AIKU_1.3B"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/AIKU_1.3B/bert-shared",
    num_train_epochs=10,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=300,
    weight_decay=0.1,
    logging_steps=50,
    report_to="wandb",
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    save_total_limit=3,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Step,Training Loss
50,8.551900
100,5.168100
150,4.548100
200,4.168600
250,4.024200
300,3.900800
350,3.654300
400,3.603200
450,3.539300
500,3.299800


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 1.5, 'no_repeat_ngram_size': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 1.5, 'no_repeat_ngram_size': 2}


TrainOutput(global_step=1480, training_loss=3.3202594241580448, metrics={'train_runtime': 954.6051, 'train_samples_per_second': 98.784, 'train_steps_per_second': 1.55, 'total_flos': 1037881512643200.0, 'train_loss': 3.3202594241580448, 'epoch': 10.0})

In [ ]:
eval_data = datasets.load_dataset("json", data_files='eval_data.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def flatten(example):
   product = example['상품']
   value = ', '.join(example['가치'])
   input = product + '<SEP>' + value
   return {'input':input}

In [ ]:
eval_dataset = eval_data.map(flatten, remove_columns=['상품','가치'])

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [ ]:
eval_dataset['train'][10]

{'input': '브라우니<SEP>달콤한'}

In [ ]:
def generate_response(test_samples, model):
    inputs = tokenizer(
        test_samples['input'],
        padding="max_length",
        truncation=True,
        max_length=ENCODER_MAX_LENGTH,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

summaries_after_tuning = generate_response(eval_dataset['train'], model)[1]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
results = []
for original, summary in zip(eval_data['train'], summaries_after_tuning):
    result = {
        "상품": original['상품'],
        "담고자 하는 가치": original["가치"],
        "카피라이팅": summary
    }
    results.append(result)

In [ ]:
results[0]

{'상품': '역사 수업', '담고자 하는 가치': ['문화적인', '과거의'], '카피라이팅': '과거를 만나다, 미래를 꿈꾸다.'}

In [ ]:
import json

output_file = 'generated_results.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)